In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Bibliotecas carregadas!")

In [ ]:
# Carregar dados processados
df = pd.read_csv('../../data/processed/vendas_tratadas.csv')
df['data_venda'] = pd.to_datetime(df['data_venda'])
df.columns = df.columns.str.lower()

print(f"📊 {len(df)} vendas carregadas!")
print(f"🌎 {df['regiao'].nunique()} regiões encontradas")
print(f"📍 Regiões: {df['regiao'].unique().tolist()}")

In [ ]:
# ANÁLISE COMPLETA POR REGIÃO
print("="*80)
print("🌎 PERFORMANCE POR REGIÃO")
print("="*80)

# Ajustado: id_produto -> id_pedido
regional = df.groupby('regiao').agg({
    'valor_total': ['sum', 'mean', 'median'],
    'quantidade': 'sum',
    'id_pedido': 'count' # Nome correto após o str.lower()
}).round(2)

# Organizando as colunas
regional.columns = ['Receita Total', 'Ticket Médio', 'Ticket Mediano', 'Qtd Produtos', 'Num Vendas']
regional = regional.sort_values('Receita Total', ascending=False)

# Adicionar participação percentual
regional['% Receita'] = (regional['Receita Total'] / regional['Receita Total'].sum() * 100).round(2)

print(regional)
print("\n")

# Destacar a região líder
regiao_lider = regional.index[0]
receita_lider = regional.loc[regiao_lider, 'Receita Total']
participacao = regional.loc[regiao_lider, '% Receita']

print(f"🥇 REGIÃO LÍDER: {regiao_lider}")
print(f"   💰 Receita: R$ {receita_lider:,.2f}")
print(f"   📊 Participação: {participacao}% do total")

In [ ]:
# ANÁLISE POR VENDEDOR (TOP 10)
print("\n" + "="*80)
print("👤 TOP 10 VENDEDORES")
print("="*80)

vendedores = df.groupby('vendedor').agg({
    'valor_total': ['sum', 'mean'],
    'id_pedido': 'count'
}).round(2)

vendedores.columns = ['Receita Total', 'Ticket Médio', 'Num Vendas']
vendedores = vendedores.sort_values('Receita Total', ascending=False).head(10)

print(vendedores)

# Vendedor destaque
vendedor_top = vendedores.index[0]
receita_vendedor = vendedores.loc[vendedor_top, 'Receita Total']
print(f"\n🥇 Vendedor destaque: {vendedor_top} - R$ {receita_vendedor:,.2f}")

In [ ]:
# MATRIZ: REGIÃO x CATEGORIA
print("\n" + "="*80)
print("📊 RECEITA POR REGIÃO E CATEGORIA")
print("="*80)

pivot_reg_cat = df.pivot_table(
    values='valor_total',
    index='regiao',
    columns='categoria',
    aggfunc='sum',
    fill_value=0
).round(2)

print(pivot_reg_cat)

# Identificar categoria mais forte por região
print("\n🎯 Categoria dominante por região:")
for regiao in pivot_reg_cat.index:
    cat_dominante = pivot_reg_cat.loc[regiao].idxmax()
    valor = pivot_reg_cat.loc[regiao, cat_dominante]
    print(f"   {regiao}: {cat_dominante} (R$ {valor:,.2f})")

In [ ]:
# DASHBOARD COMPLETO DE PERFORMANCE REGIONAL
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('🗺️ ANÁLISE DE PERFORMANCE REGIONAL', fontsize=20, fontweight='bold', y=0.995)

# Gráfico 1: Receita por Região
regional['Receita Total'].plot(kind='bar', ax=axes[0, 0], color='teal', edgecolor='black')
axes[0, 0].set_title('💰 Receita Total por Região', fontsize=14, fontweight='bold', pad=10)
axes[0, 0].set_ylabel('Receita (R$)', fontsize=11)
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=45, labelsize=10)
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1000:.0f}K'))

for i, v in enumerate(regional['Receita Total']):
    axes[0, 0].text(i, v, f'R$ {v/1000:.0f}K', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Gráfico 2: - Top 10 Vendedores
vendedores_top = df.groupby('vendedor')['valor_total'].sum().sort_values(ascending=False).head(10)
vendedores_top.sort_values().plot(kind='barh', ax=axes[0, 1], color='purple', edgecolor='black')
axes[0, 1].set_title('👤 Top 10 Vendedores - Receita', fontsize=14, fontweight='bold', pad=10)
axes[0, 1].set_xlabel('Receita (R$)', fontsize=11)
axes[0, 1].grid(axis='x', alpha=0.3)
axes[0, 1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1000:.0f}K'))

# Gráfico 3: Ticket Médio por Região
regional['Ticket Médio'].plot(kind='bar', ax=axes[1, 0], color='orange', edgecolor='black')
axes[1, 0].set_title('🎫 Ticket Médio por Região', fontsize=14, fontweight='bold', pad=10)
axes[1, 0].set_ylabel('Valor Médio (R$)', fontsize=11)
axes[1, 0].set_xlabel('')
axes[1, 0].tick_params(axis='x', rotation=45, labelsize=10)
axes[1, 0].grid(axis='y', alpha=0.3)

for i, v in enumerate(regional['Ticket Médio']):
    axes[1, 0].text(i, v, f'R$ {v:.0f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Gráfico 4: - Distribuição % de Vendas
vendas_regiao = df.groupby('regiao')['id_pedido'].count()
colors = sns.color_palette('Set2', len(vendas_regiao))
wedges, texts, autotexts = axes[1, 1].pie(
    vendas_regiao, 
    labels=vendas_regiao.index, 
    autopct='%1.1f%%',
    startangle=90, 
    colors=colors,
    textprops={'fontsize': 10, 'fontweight': 'bold'}
)
axes[1, 1].set_title('📊 Distribuição % de Vendas por Região', fontsize=14, fontweight='bold', pad=10)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)

plt.tight_layout()
plt.savefig('../graficos/nome_do_arquivo.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Dashboard salvo com sucesso!")

In [ ]:
# HEATMAP: Receita por Região e Categoria
plt.figure(figsize=(12, 6))

sns.heatmap(
    pivot_reg_cat, 
    annot=True, 
    fmt='.0f', 
    cmap='YlOrRd', 
    linewidths=0.5,
    cbar_kws={'label': 'Receita (R$)'}
)

plt.title('🔥 Mapa de Calor: Receita por Região x Categoria', 
          fontsize=16, fontweight='bold', pad=15)
plt.xlabel('Categoria', fontsize=12, fontweight='bold')
plt.ylabel('Região', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()
plt.savefig('../../images/heatmap_regiao_categoria.png', dpi=300, bbox_inches='tight')
plt.show()
plt.figure(figsize=(12, 8))
plt.savefig('../graficos/nome_do_arquivo.png', dpi=300, bbox_inches='tight')

print("✅ Heatmap salvo em: notebooks/graficos/heatmap_regiao_categoria.png")

In [ ]:
# GERAR INSIGHTS AUTOMÁTICOS
print("\n" + "="*80)
print("💡 INSIGHTS ESTRATÉGICOS - PERFORMANCE REGIONAL")
print("="*80)

# 1. Região líder
regiao_top = regional.index[0]
receita_top = regional.loc[regiao_top, 'Receita Total']
participacao_top = regional.loc[regiao_top, '% Receita']

print(f"\n🥇 REGIÃO DOMINANTE:")
print(f"   → {regiao_top} lidera com R$ {receita_top:,.2f}")
print(f"   → Representa {participacao_top}% da receita total")
print(f"   → Ticket médio: R$ {regional.loc[regiao_top, 'Ticket Médio']:,.2f}")

# 2. Região com maior ticket médio
regiao_maior_ticket = regional['Ticket Médio'].idxmax()
ticket_max = regional.loc[regiao_maior_ticket, 'Ticket Médio']

print(f"\n💰 MAIOR TICKET MÉDIO:")
print(f"   → {regiao_maior_ticket}: R$ {ticket_max:,.2f}")

# 3. Região com mais vendas
regiao_mais_vendas = regional['Num Vendas'].idxmax()
num_vendas = regional.loc[regiao_mais_vendas, 'Num Vendas']

print(f"\n📊 MAIOR VOLUME DE VENDAS:")
print(f"   → {regiao_mais_vendas}: {num_vendas:.0f} vendas")

# 4. Disparidade regional
maior_receita = regional['Receita Total'].max()
menor_receita = regional['Receita Total'].min()
disparidade = (maior_receita / menor_receita - 1) * 100

print(f"\n⚠️  DISPARIDADE REGIONAL:")
print(f"   → Diferença de {disparidade:.1f}% entre a região líder e a menor")

# 5. Potencial de crescimento
regiao_potencial = regional.index[-1]  # Região com menor receita
receita_potencial = regional.loc[regiao_potencial, 'Receita Total']

print(f"\n🎯 OPORTUNIDADE DE CRESCIMENTO:")
print(f"   → {regiao_potencial} tem o menor faturamento (R$ {receita_potencial:,.2f})")
print(f"   → Potencial de expansão e investimento")

print("\n" + "="*80)

In [ ]:
# RECOMENDAÇÕES ESTRATÉGICAS
print("\n" + "="*80)
print("🎯 RECOMENDAÇÕES ESTRATÉGICAS")
print("="*80)

# Baseado nos dados
regiao_lider = regional.index[0]
regiao_menor = regional.index[-1]
cat_forte_lider = pivot_reg_cat.loc[regiao_lider].idxmax()

print(f"\n1️⃣  EXPANSÃO:")
print(f"   → Investir em marketing na região {regiao_menor}")
print(f"   → Replicar estratégias de sucesso do {regiao_lider}")

print(f"\n2️⃣  PRODUTOS:")
print(f"   → Categoria '{cat_forte_lider}' forte no {regiao_lider}")
print(f"   → Considerar expandir portfólio similar em outras regiões")

print(f"\n3️⃣  TICKET MÉDIO:")
if regiao_maior_ticket != regiao_lider:
    print(f"   → {regiao_maior_ticket} tem alto ticket médio mas menor volume")
    print(f"   → Oportunidade de aumentar conversão mantendo qualidade")

print(f"\n4️⃣  DISTRIBUIÇÃO:")
print(f"   → Avaliar logística para regiões com menor presença")
print(f"   → Parcerias locais podem reduzir custos")

print("\n" + "="*80)

In [ ]:
# Definindo o caminho para a pasta de gráficos (dentro de notebooks)
caminho_graficos = '../graficos/dashboard_regional.png'

# Salvando na pasta de gráficos
plt.savefig(caminho_graficos, dpi=300, bbox_inches='tight')

# Mantendo o salvamento na pasta de imagens geral para o relatório final
plt.savefig('../graficos/nome_do_arquivo.png', dpi=300, bbox_inches='tight')

print(f"✅ Dashboard salvo em: notebooks/graficos/dashboard_regional.png")

In [ ]:
import pandas as pd
import sqlite3

# Conectar ao banco
conn = sqlite3.connect('../../sql/vendas_database.db')

# Exemplo de Query SQL: Ranking de Vendedores por Receita
query = """
SELECT 
    vendedor, 
    SUM(valor_total) as receita_total,
    COUNT(id_pedido) as total_vendas
FROM vendas
GROUP BY vendedor
ORDER BY receita_total DESC
LIMIT 5
"""

# Ler o resultado do SQL direto para um DataFrame
df_sql = pd.read_sql_query(query, conn)
conn.close()

print("📊 Resultado via Query SQL:")
df_sql

In [ ]:
import sqlite3
import pandas as pd

# Conectando ao seu banco de dados na pasta sql
conn = sqlite3.connect('../../sql/vendas_database.db')

# Esta query faz o cálculo matemático da Curva ABC
query_vendedores_abc = """
WITH faturamento_vendedor AS (
    SELECT 
        vendedor,
        SUM(valor_total) as receita_vendedor
    FROM vendas
    GROUP BY vendedor
),
acumulado_vendedor AS (
    SELECT 
        vendedor,
        receita_vendedor,
        SUM(receita_vendedor) OVER (ORDER BY receita_vendedor DESC) as receita_acumulada,
        SUM(receita_vendedor) OVER () as receita_total_geral
    FROM faturamento_vendedor
)
SELECT 
    vendedor,
    receita_vendedor,
    ROUND((receita_acumulada / receita_total_geral) * 100, 2) as pct_acumulado,
    CASE 
        WHEN (receita_acumulada / receita_total_geral) <= 0.80 THEN 'A (Top)'
        WHEN (receita_acumulada / receita_total_geral) <= 0.95 THEN 'B (Médio)'
        ELSE 'C (Baixo)'
    END as categoria_vendedor
FROM acumulado_vendedor
ORDER BY receita_vendedor DESC;
"""

# Executa a query e transforma em uma tabela (DataFrame)
df_vendedores_abc = pd.read_sql_query(query_vendedores_abc, conn)
conn.close()

# Mostra o resultado na tela do VS Code
df_vendedores_abc

# 📈 Insights da Análise de Performance
Após processar os dados de 2025, identificamos os seguintes pontos-chave:

### 1. Curva ABC de Vendedores
* *Classe A:* Um pequeno grupo de vendedores é responsável por 80% do faturamento. 
* *Ação sugerida:* Entender as táticas desses vendedores para replicar no restante do time.

### 2. Concentração Regional
* A região *[insira a região líder aqui]* domina as vendas, enquanto a região *[insira a menor aqui]* possui o maior potencial de expansão.

### 3. Mix de Produtos
* Identificamos que a categoria *[categoria top]* possui o melhor Ticket Médio, sendo a mais rentável para a empresa.
*

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Criando o gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(data=df_vendedores_abc, x='vendedor', y='receita_vendedor', hue='categoria_vendedor')

# Formatando o gráfico
plt.title('Performance dos Vendedores por Categoria ABC', fontsize=14)
plt.xticks(rotation=45)
plt.ylabel('Receita Total (R$)')
plt.xlabel('Vendedor')

# Salvando na sua pasta de gráficos
plt.savefig('../graficos/nome_do_arquivo.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Conectar ao banco na pasta sql
conn = sqlite3.connect('../../sql/vendas_database.db')

# 2. Query para agrupar faturamento por mês
query_mensal = """
SELECT 
    strftime('%m', data_venda) as mes,
    SUM(valor_total) as faturamento
FROM vendas
GROUP BY mes
ORDER BY mes
"""

df_mensal = pd.read_sql_query(query_mensal, conn)
conn.close()

# 3. Criar o gráfico de linha para ver a tendência
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_mensal, x='mes', y='faturamento', marker='o', color='green', linewidth=2.5)

plt.title('Evolução do Faturamento Mensal - 2025', fontsize=14, fontweight='bold')
plt.xlabel('Mês do Ano')
plt.ylabel('Receita (R$)')
plt.grid(True, alpha=0.3)

# Salvar na pasta correta
plt.savefig('../graficos/sazonalidade_mensal.png', bbox_inches='tight')
plt.show()
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Conectar ao banco na pasta sql
conn = sqlite3.connect('../../sql/vendas_database.db')

# 2. Query para agrupar faturamento por mês
query_mensal = """
SELECT 
    strftime('%m', data_venda) as mes,
    SUM(valor_total) as faturamento
FROM vendas
GROUP BY mes
ORDER BY mes
"""

df_mensal = pd.read_sql_query(query_mensal, conn)
conn.close()

# 3. Criar o gráfico de linha para ver a tendência
plt.figure(figsize=(12, 5))
sns.lineplot(data=df_mensal, x='mes', y='faturamento', marker='o', color='green', linewidth=2.5)

plt.title('Evolução do Faturamento Mensal - 2025', fontsize=14, fontweight='bold')
plt.xlabel('Mês do Ano')
plt.ylabel('Receita (R$)')
plt.grid(True, alpha=0.3)

# Salvar na pasta correta
plt.savefig('../graficos/nome_do_arquivo.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Exportando os resultados para CSV/Excel para uso externo
df_vendedores_abc.to_csv('../../data/processed/vendedores_abc.csv', index=False)
df_mensal.to_csv('../../data/processed/faturamento_mensal.csv', index=False)

print("💾 Tabelas exportadas com sucesso para a pasta data/processed!")